In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostClassifier

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from utils import *
import os
print(os.listdir("../input/features/"))
SEED=2019

['train_y.pkl', 'train_uid_features.pkl', 'train_features.pkl', 'useful_features.npy', 'test_features.pkl', 'test_uid_features.pkl', 'test.pkl', 'train.pkl']


In [2]:
train = pd.read_pickle('../input/features/train_features.pkl')
test= pd.read_pickle('../input/features/test_features.pkl')
train_y = pd.read_pickle('../input/features/train_y.pkl')


print(train.shape)
print(train_y.shape)
print(test.shape)

(590540, 289)
(590540,)
(506691, 289)


In [3]:
# train_uid = pd.read_pickle('../input/features/train_uid_features.pkl')
# test_uid= pd.read_pickle('../input/features/test_uid_features.pkl')
# train = train.merge(train_uid, how='left', left_index=True, right_index=True)
# test = test.merge(test_uid, how='left', left_index=True, right_index=True)
# print(train_uid.shape)
# print(test_uid.shape)

In [4]:
# features_columns = np.load('../input/features/useful_features.npy', allow_pickle=True)
rm_cols = []
features_columns = list(train)
for col in rm_cols:
    if col in features_columns:
        features_columns.remove(col)
        
# features_columns = np.array(list(features_columns) + list (train_uid.columns.values))

In [5]:
########################### Model params
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'n_estimators':2000,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': SEED,
                    'early_stopping_rounds':100, 
                } 

In [6]:
# # Negative downsampling
# train['isFraud'] = train_y

# train_pos = train[train['isFraud']==1]
# train_neg = train[train['isFraud']==0]

# train_neg = train_neg.sample(int(train.shape[0] * 0.5), random_state=42)
# train = pd.concat([train_pos,train_neg]).sort_index()
# train_y = train['isFraud']
# train = train.drop('isFraud',axis=1)

# print(train.shape)
# print(train_y.shape)
# print(test.shape)

In [9]:
%%time
test_predictions, auc_score = make_predictions(train, test, train_y, features_columns, lgb_params)

Fold: 0  -  453219 137321
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.947765	valid_1's auc: 0.882223
[200]	training's auc: 0.972021	valid_1's auc: 0.894509
[300]	training's auc: 0.986585	valid_1's auc: 0.901049
[400]	training's auc: 0.993629	valid_1's auc: 0.905189
[500]	training's auc: 0.996825	valid_1's auc: 0.907438
[600]	training's auc: 0.998438	valid_1's auc: 0.909108
[700]	training's auc: 0.999219	valid_1's auc: 0.9106
[800]	training's auc: 0.999594	valid_1's auc: 0.911565
[900]	training's auc: 0.999792	valid_1's auc: 0.912777
[1000]	training's auc: 0.999893	valid_1's auc: 0.913404
[1100]	training's auc: 0.999946	valid_1's auc: 0.913678
Early stopping, best iteration is:
[1078]	training's auc: 0.999937	valid_1's auc: 0.913807
     Value                         Feature
0        5                              M1
1       10                    addr2_fq_enc
2       30                           addr2
3       48        card3_TransactionAmt_std


Fold: 1  -  488908 101632
Training until validation scores don't improve for 100 rounds.


KeyboardInterrupt: 

In [8]:
test_predictions.to_csv('../submissions/sub.csv', index=True) # LB 0.9495

NameError: name 'test_predictions' is not defined

In [ ]:
#[0.9411, 0.9467] 0.0056
#[0.9425, 0.9485] 0.0069
#[0.9382, 0.9433] 0.0051
#[0.9443, 0.9495] 0.0052
#[0.9434, 0.9477] 0.0043
#[0.9432, 0.9481] 0.0051
# [0.939, 0.9437] baseline

In [ ]:
train.head()

In [ ]:
# Fold: 0  -  453219 137321
# Training until validation scores don't improve for 100 rounds.
# [2000]	training's auc: 0.998268	valid_1's auc: 0.914522
# [4000]	training's auc: 0.999969	valid_1's auc: 0.920546
# Early stopping, best iteration is:
# [4223]	training's auc: 0.999981	valid_1's auc: 0.92091
# Fold: 1  -  488908 101632
# Training until validation scores don't improve for 100 rounds.
# [2000]	training's auc: 0.998308	valid_1's auc: 0.943141
# [4000]	training's auc: 0.999969	valid_1's auc: 0.946945
# Early stopping, best iteration is:
# [4796]	training's auc: 0.999995	valid_1's auc: 0.947414
# Fold: 2  -  497955 92585
# Training until validation scores don't improve for 100 rounds.
# [2000]	training's auc: 0.998046	valid_1's auc: 0.945959
# [4000]	training's auc: 0.999959	valid_1's auc: 0.950484
# Did not meet early stopping. Best iteration is:
# [5000]	training's auc: 0.999996	valid_1's auc: 0.951029
# Fold: 3  -  501214 89326
# Training until validation scores don't improve for 100 rounds.
# [2000]	training's auc: 0.998041	valid_1's auc: 0.940237
# [4000]	training's auc: 0.999954	valid_1's auc: 0.943445
# Early stopping, best iteration is:
# [4070]	training's auc: 0.999961	valid_1's auc: 0.943516
# Fold: 4  -  504519 86021
# Training until validation scores don't improve for 100 rounds.
# [2000]	training's auc: 0.997872	valid_1's auc: 0.950521
# Early stopping, best iteration is:
# [3389]	training's auc: 0.999838	valid_1's auc: 0.952638
# Fold: 5  -  506885 83655
# Training until validation scores don't improve for 100 rounds.
# [2000]	training's auc: 0.997762	valid_1's auc: 0.955094
# [4000]	training's auc: 0.999939	valid_1's auc: 0.958693
# Early stopping, best iteration is:
# [4520]	training's auc: 0.999979	valid_1's auc: 0.958905
# ---------------------------------------
# OOF AUC: 0.9443314174220738

In [ ]:
# params = {'num_leaves': 491,
#           'min_child_weight': 0.03454472573214212,
#           'feature_fraction': 0.3797454081646243,
#           'bagging_fraction': 0.4181193142567742,
#           'min_data_in_leaf': 106,
#           'objective': 'binary',
#           'max_depth': -1,
#           'learning_rate': 0.006883242363721497,
#           "boosting_type": "gbdt",
#           "bagging_seed": 11,
#           "metric": 'auc',
#           "verbosity": -1,
#           'reg_alpha': 0.3899927210061127,
#           'reg_lambda': 0.6485237330340494,
#           'random_state': 47
#          }

In [ ]:
# from sklearn.feature_selection import RFECV
# train.fillna(-999, inplace=True)
# clf = lgb.LGBMClassifier(**params)
# rfe = RFECV(estimator=clf, step=10, cv=KFold(n_splits=5, shuffle=False), scoring='roc_auc', verbose=2)

# rfe.fit(train, train_y)

# print('Optimal number of features:', rfe.n_features_)

In [ ]:
# plt.figure(figsize=(14, 8))
# plt.xlabel("Number of features selected")
# plt.ylabel("Cross validation score")
# plt.plot(range(1, len(rfe.grid_scores_) + 1), rfe.grid_scores_)
# plt.show()

In [ ]:
# for col in train.columns[rfe.ranking_ == 1]:
#     print(col)